In [1]:
import numpy
import numpy as np
import scipy

from pandas import DataFrame

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt


/root/p3/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/root/p3/lib/python3.4/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%matplotlib inline

<h2>1) Скачаем датасет</h2>

In [3]:
! wget http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip
! unzip smsspamcollection.zip


--2017-05-17 19:52:10--  http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip
Resolving www.dt.fee.unicamp.br (www.dt.fee.unicamp.br)... 143.106.12.20
Connecting to www.dt.fee.unicamp.br (www.dt.fee.unicamp.br)|143.106.12.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210521 (206K) [application/zip]
Saving to: ‘smsspamcollection.zip.1’

100%[======================================>] 210 521     79,4KB/s   in 2,6s   

2017-05-17 19:52:14 (79,4 KB/s) - ‘smsspamcollection.zip.1’ saved [210521/210521]

Archive:  smsspamcollection.zip
replace readme? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


<h2>2,3) Считаем данные и подготовим списки признаков</h2>

In [31]:
src = open('SMSSpamCollection.txt')
items = [line.strip('\n').split('\t')[::-1] for line in src.readlines()]
texts = [it[0] for it in items]
labels = [it[1] for it in items]
for i, label in enumerate(labels):
    labels[i] = 1 if label == 'spam' else 0

<h2>4) получим признаки bag of words</h2>

In [32]:
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(texts)

<h2>5) Подготовим score_function</h2>

In [33]:
def score_func(est, X, y):
    cv = cross_val_score(est, X, y, cv=5, scoring='average_precision')
    # print(cv)
    return cv.mean()

<h2>проверим на LogisticRegression</h2>

In [34]:
print(score_func(LogisticRegression(), features, labels))

0.973318472579


<h2>Ответ в п.5 0.933348526858</h2>

<h2>6) Отклассифицируем данные сообщения классификатором, обученным на всей выборке</h2>

In [35]:
x = ["FreeMsg: Txt: CALL to No:86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
     "FreeMsg: Txt: claim your reward of 3 hours talk time",
     "Have you visited the last lecture on physics?", 
     "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", 
     "Only 99$"]

In [36]:
new_labels = LogisticRegression().fit(features, labels).predict(vectorizer.transform(x))
print(new_labels)

[1 1 0 0 0]


<h2>Ответ на п.6: "1 1 0 0 0"</h2>

<h2>7) Попробуем разные ngram_range</h2>

In [37]:
ngram_ranges = [ (2, 2), (3, 3), (1, 3)]

In [38]:
for ngram_range in ngram_ranges:
    print(score_func(LogisticRegression(),
                     CountVectorizer(ngram_range=ngram_range).fit_transform(texts),
                     labels
                     ))

0.95020180128
0.898932803618
0.975194915229


<h2>Ответ в п.7: "0.82 0.73 0.93"</h2>

<h2>8) То же самое для MultinomialNB()</h2>

In [39]:
for ngram_range in ngram_ranges:
    print(score_func(MultinomialNB(),
                     CountVectorizer(ngram_range=ngram_range).fit_transform(texts),
                     labels
                     ))

0.963906947377
0.866417933548
0.979131556061


<h2>Ответ в п.8: "0.65 0.39 0.89"</h2>

<h2> 9) Попробуем TfidfVectorizer + LogisticRegression</h2>

In [40]:
for ngram_range in ngram_ranges:
    print(score_func(LogisticRegression(),
                     TfidfVectorizer(ngram_range=ngram_range).fit_transform(texts),
                     labels
                     ))

0.94377493097
0.89870880652
0.971449234172


<h2> :( Из-за маленькой тренировочной выборки, многие стоп-слова получают большой вес</h2>

<h2> 10) Попробуем улучшить score </h2>

<h3> Заменим логистическую регрессию(один нейрон) на полносвязную нейросеть </h3>

In [41]:
import random

best_score = 0
best_layers = (1, )
for i in range(100):
    n_layers = random.randint(1, 4) # от 1 до 4 слоев
    layers = []
    for layer in range(n_layers):
        layers.append(random.randint(1, 8)) # от 1 до 8 нейронов в слое
    classifier = MLPClassifier(solver='lbfgs', hidden_layer_sizes=layers, random_state=1, activation='relu')
    score = score_func(classifier, features, labels)
    if score > best_score:
        best_score = score
        best_layers = layers
        print("new best score %f" % score)
    if i % 10 == 0:
        print(i, "iterations passed...")


In [44]:
print(best_score, best_layers)

0.968315480645 [6]


<h4> "Затащила" однослойная сеть из 6 нейронов </h4>

<h3> Попробуем понизить размерность <h3>

In [ ]:
svd = TruncatedSVD(1000)
svd.fit(features)

plt.figure(1, figsize=(4, 3))
plt.clf()
plt.axes([.2, .2, .7, .7])
plt.plot(svd.explained_variance_, linewidth=2)
plt.axis('tight')
plt.xlabel('n_components')
plt.ylabel('explained_variance_')

In [ ]:
N_COMP = 500

In [ ]:
svd = TruncatedSVD(N_COMP)
logistic = LogisticRegression(n_jobs=-1, class_weight='balanced')

In [ ]:
pipe = Pipeline(steps=[('svd', svd), ('logistic', logistic)])
Cs = np.logspace(-2, 2, 20)
estimator = GridSearchCV(pipe, dict(logistic__C=Cs), n_jobs=-1, scoring=score_func)


In [ ]:
estimator.fit(features, labels)

In [ ]:
b_est = estimator.best_estimator_
b_params = estimator.best_params_
b_score = estimator.best_score_

In [ ]:
print(estimator.param_grid)

In [ ]:
score_func(b_est, features, labels)

<h3> А теперь сравним с тем же, но без понижения размерности </h3>

In [ ]:
logistic = LogisticRegression(n_jobs=-1, class_weight='balanced')
estimator_2 = GridSearchCV(logistic, dict(C=Cs), n_jobs=-1, scoring=score_func)
estimator_2.fit(features, labels)

In [ ]:
b_est2 = estimator_2.best_estimator_
b_params2 = estimator_2.best_params_
b_score2 = estimator_2.best_score_
print(b_params2, b_score2)

In [ ]:
plt.plot(Cs, [x[1] for x in estimator_2.grid_scores_])

In [ ]:
Cs = np.linspace(5, 10, 100)
logistic = LogisticRegression(n_jobs=-1, class_weight='balanced')
estimator_2 = GridSearchCV(logistic, dict(C=Cs), n_jobs=-1, scoring=score_func)
estimator_2.fit(features, labels)

In [ ]:
b_est2 = estimator_2.best_estimator_
b_params2 = estimator_2.best_params_
b_score2 = estimator_2.best_score_
print(b_params2, b_score2)

In [ ]:
plt.plot(Cs, [x[1] for x in estimator_2.grid_scores_])

In [ ]:
features_mixed_case = CountVectorizer(lowercase=False).fit_transform(texts)

In [ ]:
Cs = np.linspace(0.1, 10, 100)
logistic = LogisticRegression(n_jobs=-1, class_weight='balanced')
estimator_2 = GridSearchCV(logistic, dict(C=Cs), n_jobs=-1, scoring=score_func)
estimator_2.fit(features, labels)

In [ ]:
b_est2 = estimator_2.best_estimator_
b_params2 = estimator_2.best_params_
b_score2 = estimator_2.best_score_
print(b_params2, b_score2)

In [ ]:
score_func(b_est2, features, labels)

In [ ]:
score_func(LogisticRegression(), features_mixed_case, labels)

<h3><p>Применение других классификаторов вместо логистической регрессии повысить скор не помогло</p>
<p>В среднем, линейные классификаторы дали скор ~90%</p><h3>

<h2> 11) Выводы:</h2>
<h4><p> 1. Tfidf нужна большая выборка для обучения, на малой мусорные слова могут приобретать большой вес. </p>
<p> 2. Соответственно TfidfVectorizer не всегда лучше, чем CountVectorizer </p>
<p> 3. Иногда можно уменьшать размерность используя max_features у vectorizer'a вместо PCA, SVD, особо не теряя в качестве, но значительно выигрывая в скорости </p>
<p> 3. Байесовским методам нужна значительно бОльшая выборка, чем в этом случае. </p>
<p> 4. При малых размерах выборки опираться на коллокации неэффективныо</p>
<p> 5. Используя одинаковое решение можно получить разные результаты на кросс-валидации </p>
</h4>